In [1]:
!pip install --upgrade --force-reinstall scikit-learn==1.3.2 scikeras numpy==1.24.3 tensorflow==2.14 --quiet

# AFTER running this cell, go to Runtime > Restart runtime.
# Then run the rest of the notebook from the top.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
google-colab 1.0.0 requires google-auth==2.38.0, but you have google-auth 2.40.0 which is incompatible.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.7 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompa

In [2]:
# STEP 0 — INSTALL & VERSION FIX
!pip install --upgrade --force-reinstall scikit-learn==1.3.2 scikeras numpy==1.24.3 --quiet

# AFTER running this cell, go to Runtime > Restart runtime.
# Then run the rest of the notebook from the top.

# STEP 1 — IMPORT LIBRARIES
import pandas as pd
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import pickle

# STEP 2 — LOAD & CLEAN DATA
data = pd.read_csv('/content/Data (1).csv')  # Upload your CSV to Colab
data = data[['text', 'sentiment']]
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply(lambda x: re.sub('[^a-zA-z0-9\s]', '', x))

for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

# STEP 3 — TOKENIZATION
max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

# STEP 4 — LABEL ENCODING
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)

# STEP 5 — TRAIN/TEST SPLIT
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# STEP 6 — BUILD AND TRAIN MODEL
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_features, embed_dim, input_length=X.shape[1]))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', patience=2)

model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2, callbacks=[early_stop])

# STEP 7 — SAVE MODEL, TOKENIZER & ENCODER
model.save('/content/sentiment_model.h5')

with open('/content/tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

with open('/content/labelencoder.pkl', 'wb') as f:
    pickle.dump(labelencoder, f)

print("✅ Model, tokenizer, and label encoder saved successfully!")

# STEP 8 — PREDICT ON NEW TEXT
from tensorflow.keras.models import load_model

model = load_model('/content/sentiment_model.h5')

with open('/content/tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

with open('/content/labelencoder.pkl', 'rb') as f:
    labelencoder = pickle.load(f)

new_text = ["A lot of good things are happening. We are respected again throughout the world, and that's a great thing .@realDonaldTrump"]

new_text_seq = tokenizer.texts_to_sequences(new_text)
new_text_pad = pad_sequences(new_text_seq, maxlen=X.shape[1])

prediction = model.predict(new_text_pad)
predicted_label = labelencoder.inverse_transform([prediction.argmax()])

print(f"Prediction: {predicted_label[0]}")

# STEP 9 — GRIDSEARCHCV
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV

X_train_gs, X_test_gs, y_train_gs, y_test_gs = train_test_split(X, integer_encoded, test_size=0.2, random_state=42)

def create_model(embed_dim=128, lstm_out=196):
    m = Sequential()
    m.add(Embedding(max_features, embed_dim, input_length=X.shape[1]))
    m.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    m.add(Dense(3, activation='softmax'))
    m.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return m

model_wrapper = KerasClassifier(model=create_model, verbose=0)

param_grid = {
    'embed_dim': [64, 128],
    'lstm_out': [100, 196],
    'batch_size': [32],
    'epochs': [3]
}

grid = GridSearchCV(estimator=model_wrapper, param_grid=param_grid, cv=3)
grid_result = grid.fit(X_train_gs, y_train_gs)

print("✅ Best parameters found:")
print(grid_result.best_params_)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.18.0 requires tensorflow<2.19,>=2.18, but you have tensorflow 2.14.0 which is incompatible.
tensorstore 0.1.74 requires ml_dtypes>=0.3.1, but you have ml-dtypes 0.2.0 which is incompatible.
ydf 0.11.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 4.25.7 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.3 which is incompatible.
tensorflow-text 2.18.1 requires tensorflow<2.19,>=2.18.0, but you have tensorflow 2.14.0 which is incompatible.
pymc 5.22.0 requires numpy>=1.25.0, but you have numpy 1.24.3 which is incompatible.
langchain-core 0.3.56 requires packaging<25,>=23.2, but you have packaging 25.0 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.24.3 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have

AttributeError: module 'numpy' has no attribute 'dtypes'